# Módulos

In [4]:
import os # Arquivos
import pandas as pd # Dados

# Modelos
from sklearn.feature_extraction.text import CountVectorizer # Tokenização
from sklearn.decomposition import LatentDirichletAllocation # LDA
from sklearn.model_selection import KFold # Cross-validation

import multiprocessing as mp # Magia negra (paralelização)
from time import perf_counter # Cronômetro

# Funções

In [5]:
def estimar_perplexidades_com_KFold(corpus, 
                                    learning_method, 
                                    n_components, 
                                    doc_topic_prior = None, topic_word_prior = None,
                                    learning_decay = .75, learning_offset = None, batch_size = None,
                                    n_splits = 5, random_state = 49, n_jobs = mp.cpu_count()-1):

    kf = KFold(n_splits = n_splits, shuffle = True, random_state = random_state)

    modelos, resultados = [], []
    for iteração, (índices_de_treino, índices_de_teste) in enumerate(kf.split(corpus)):

        # Iteração
        iteração += 1
        print(f'K{iteração}')

        # Divisão entre treino e teste
        corpus_de_treino, corpus_de_teste = corpus[índices_de_treino], corpus[índices_de_teste]

        # Latent Dirichlet Allocation
        if learning_method == 'online':
            n_docs = corpus.shape[0] # Tamanho do corpus para parâmetros ajustáveis...
            learning_offset =  int(n_docs*0.001) if learning_offset == None else learning_offset
            batch_size =  max(128, int(n_docs*0.01)) if batch_size == None else batch_size
            
            modelo_de_LDA = LatentDirichletAllocation(n_components = n_components, 
                                                      learning_method = learning_method, 
                                                      doc_topic_prior = doc_topic_prior, topic_word_prior = topic_word_prior,
                                                      learning_decay = learning_decay, learning_offset = learning_offset, batch_size = batch_size,
                                                      random_state = random_state, n_jobs = n_jobs)
        else: # 'batch'
            modelo_de_LDA = LatentDirichletAllocation(n_components = n_components, 
                                                      learning_method = learning_method, 
                                                      doc_topic_prior = doc_topic_prior, topic_word_prior = topic_word_prior,
                                                      random_state = random_state, n_jobs = n_jobs)
            
        modelo_de_LDA.fit(corpus_de_treino)
        modelos.append(modelo_de_LDA)

        # Perplexidade e score do modelo 
        perplexidade, pontuação = modelo_de_LDA.perplexity(corpus_de_teste), modelo_de_LDA.score(corpus_de_teste)
        
        print('>>> Perplexidade:', round(perplexidade,2))
        print('>>> Pontuação:', round(pontuação,2),'\n')
        
        resultados.append([iteração, perplexidade, pontuação])

    # Output, enfim
    return modelos, resultados

# Estimação

## Parâmetros

In [8]:
diretório = os.getcwd() 
diretório_anterior = '\\'.join(diretório.split('\\')[:-1])
diretório_dos_corpus = diretório_anterior + '\\' + '1. Coleta e tratamento\\bases para o LDA'

seleções = ['valor', 'globorural', 'epocanegocios', 'pipelinevalor', 'umsoplaneta', 'revistapegn', 'valorinveste']
#seleções += [f'agregação {i}' for i in range(1,5+1)]

números_por_testar_por_seleção = {

    'valor': (1,10,20,30,40,50,60,70,80),
    'valorinveste': (1,10,20,30,40,50,60,70,80),
    'pipelinevalor': (1,2,3,4,5,7,9,11,13,15),
    'globorural': (1,10,20,30,40,50,60,70,80),
    'epocanegocios': (1,10,20,30,40,50,60,70,80),
    'revistapegn': (1,10,20,30,40,50,60,70,80),
    'umsoplaneta': (1,2,3,4,5,7,9,11,13,15),
        
}

for i in range(1,5+1):
    números_por_testar_por_seleção[f'agregação {i}'] = (1,20,40,50,60,70,80,100,120)

métodos = ('online', 'batch')
diretório_dos_resultados = 'perplexidades'
colunas_ordenadas = ['Método', 'Nº de tópicos', 'Iteração', 'Perplexidade', 'Pontuação']

## Perplexidades

In [9]:
for método_de_aprendizado in métodos:
    for seleção in seleções:
        
        if método_de_aprendizado == 'batch' and 'agregação' in seleção:
            continue # Mucho texto rodar essas em batch...
            
        for recorte in ['total']:
            print(f'-------\nLendo "{seleção} - {recorte}" com {método_de_aprendizado}!\n')
            
            arquivo_do_corpus = f'{seleção} - {recorte}.parquet'
            dataframe = pd.read_parquet(diretório_dos_corpus + '\\' + arquivo_do_corpus)
            df = dataframe[dataframe.corpus != ''].copy()
            
            vetorizador = CountVectorizer()
            corpus = vetorizador.fit_transform(df.corpus)
    
            for número_de_tópicos in números_por_testar_por_seleção[seleção]:
                arquivo_de_resultados = seleção + ' - ' + recorte + ' - ' + método_de_aprendizado + ' - ' + str(número_de_tópicos) + ' - tópicos.xlsx'
                if arquivo_de_resultados in os.listdir(diretório_dos_resultados):
                    print(f'---\nJá lido: {número_de_tópicos} tópico(s) em "{método_de_aprendizado}"... Pulando!\n')
                    continue # Para pular os já calculados!
    
                print(f'---\nNº de tópicos: {número_de_tópicos}.')
                print(f'Método: {método_de_aprendizado}.\n')
                
                modelos, resultados = estimar_perplexidades_com_KFold(corpus = corpus, 
                                                                      learning_method = método_de_aprendizado,
                                                                      n_components = número_de_tópicos)
        
                resultados = pd.DataFrame(resultados, columns = ['Iteração', 'Perplexidade', 'Pontuação'])
                resultados['Método'], resultados['Nº de tópicos']  = método_de_aprendizado, número_de_tópicos
        
                resultados[colunas_ordenadas].to_excel(f'{diretório_dos_resultados}/{arquivo_de_resultados}', index = False)
    
                print('Informações armazenadas com sucesso!\n')

-------
Lendo "valor - total" com online!

---
Já lido: 1 tópico(s) em "online"... Pulando!

---
Já lido: 10 tópico(s) em "online"... Pulando!

---
Já lido: 20 tópico(s) em "online"... Pulando!

---
Já lido: 30 tópico(s) em "online"... Pulando!

---
Já lido: 40 tópico(s) em "online"... Pulando!

---
Já lido: 50 tópico(s) em "online"... Pulando!

---
Já lido: 60 tópico(s) em "online"... Pulando!

---
Já lido: 70 tópico(s) em "online"... Pulando!

---
Já lido: 80 tópico(s) em "online"... Pulando!

-------
Lendo "globorural - total" com online!

---
Já lido: 1 tópico(s) em "online"... Pulando!

---
Já lido: 10 tópico(s) em "online"... Pulando!

---
Já lido: 20 tópico(s) em "online"... Pulando!

---
Já lido: 30 tópico(s) em "online"... Pulando!

---
Já lido: 40 tópico(s) em "online"... Pulando!

---
Já lido: 50 tópico(s) em "online"... Pulando!

---
Já lido: 60 tópico(s) em "online"... Pulando!

---
Já lido: 70 tópico(s) em "online"... Pulando!

---
Já lido: 80 tópico(s) em "online"... Pul